<a href="https://colab.research.google.com/github/orca3/llm-model-serving/blob/main/ch02/workthrough_LLM_execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>